In [1]:
# The European Union has 27 member states
# The site use GB to denote UK_ASOS
# the eu_member_codes has 28 variables: UK + EU

import os
import requests
import pandas as pd
import datetime
import time
import concurrent.futures
from bs4 import BeautifulSoup
from tqdm import tqdm

In [2]:
eu_member_codes = [
    'AT', 'BE', 'BG', 'HR', 'CY', 'CZ', 'DK', 'EE', 'FI', 'FR',
    'DE', 'GR', 'HU', 'IE', 'IT', 'LV', 'LT', 'LU', 'MT', 'NL',
    'PL', 'PT', 'RO', 'SK', 'SI', 'ES', 'SE', 'GB',
]

# Extract data time range
# startts = datetime.datetime(2022, 1, 1)
# endts = datetime.datetime(2023, 8, 2)

# Number of attempts to download data
MAX_ATTEMPTS = 6

In [3]:
def get_all_network():
    url = "https://mesonet.agron.iastate.edu/request/download.phtml?network=FR__ASOS"
    response = requests.get(url)
    soup = BeautifulSoup(response.content, "html.parser")
    select_element = soup.find("select")
    if select_element:
        option_elements = select_element.find_all("option")

        option_values = [option["value"] for option in option_elements if option.get("value")]

        print("Option values:")
        for value in option_values:
            print(value)
    else:
        print("Select element not found on the page.")

In [4]:
def get_network_url(country_list):
    valid_urls = []
    invalid_urls = []

    # for i in eu_member_codes:
    for i in country_list:
        url_network = f"https://mesonet.agron.iastate.edu/request/download.phtml?network={i}__ASOS"
        response = requests.head(url_network)

        if response.status_code == 200:
            valid_urls.append(url_network)
        else:
            invalid_urls.append(f"Invalid URL: {url_network}")

    return valid_urls

In [5]:
def get_current_directory():
    if "__file__" in globals():
        # Running in a Python file
        return os.path.abspath(os.path.dirname(__file__))
    else:
        # Running in a Jupyter Notebook
        return os.path.abspath(os.path.dirname(""))


In [6]:
def create_folder(c):
    current_directory = get_current_directory()
    folder_name = f"{c}_ASOS_DATA"
    folder_path = os.path.join(current_directory, folder_name)

    try:
        os.mkdir(folder_path)
        print(f"Folder '{folder_path}' created successfully.")
    except FileExistsError:
        print(f"Folder '{folder_path}' already exists.")

    return folder_path


In [7]:
def get_all_station_by_network(country_list):
    # valid_urls = get_network_url()
    for i in country_list:
        url_station_geojson = (
            f"https://mesonet.agron.iastate.edu/geojson/network/{i}__ASOS.geojson"
        )
        output_directory = create_folder(i)
        output_filename = f"{i}__asos_station_network.csv"
        output_filepath = os.path.join(output_directory, output_filename)

        # Get GeoJSON data
        response = requests.get(url_station_geojson)
        geojson_data = response.json()
        #  Create a list to store geojson
        data = []

        # Extract GeoJSON items
        for feature in geojson_data["features"]:
            properties = feature["properties"]
            geometry = feature["geometry"]
            row = {
                # "Type": feature.get("type", None),
                "Name": properties.get("sname", None),
                "ID": feature.get("id", None),
                "Latitude": geometry.get("coordinates", None)[1],
                "Logitude": geometry.get("coordinates", None)[0],
                "Elevation": properties.get("elevation", None),
                "Country": properties.get("country", None),
                "Network": properties.get("network", None),
                "Archive_Begin": properties.get("archive_begin", None),
                "Archive_End": properties.get("archive_end", None),
                "Time_Domain": properties.get("time_domain", None),
                # "Properties": properties
            }
            data.append(row)

        # Transfer the list to Pandas DataFrame
        df = pd.DataFrame(data)
        df.to_csv(output_filepath, index=False, encoding="utf-8")

    return df

In [9]:
def get_data_url(df, startts, endts):
    url_site_list = []
    id_list = []
    url_site_header = "https://mesonet.agron.iastate.edu/cgi-bin/request/asos.py?"
    url_site_tail = f"data=tmpc&"  # add all data variables f"data=tmpf&"
    url_site_tail += startts.strftime("year1=%Y&month1=%m&day1=%d&")  # add start date
    url_site_tail += endts.strftime("year2=%Y&month2=%m&day2=%d&")  # add end date
    url_site_tail += f"tz=Etc%2FUTC&format=onlycomma&latlon=no&elev=no&missing=null&trace=T&direct=no&report_type=3&report_type=4"  # add data format
    for id in df["ID"]:
        url_site = f"{url_site_header}station={id}&{url_site_tail}"  # add all stations
        url_site_list.append(url_site)
        id_list.append(id)

    return url_site_list, id_list  # return station list

In [10]:
def download_data(url_data):
    attempt = 0
    while attempt < MAX_ATTEMPTS:
        try:
            response = requests.get(url_data, timeout=300)
            data = response.text
            if data is not None and not data.startswith("ERROR"):
                return data
        except Exception as e:
            print(f"download_data({url_data}) failed with {e}")
            time.sleep(5)
        attempt += 1

    print("Exhausted attempts to download, returning empty data")
    return ""

In [11]:
def save_data(url_site, country, station, startts, endts):
    data = download_data(url_site)
    # output_filename = f"{country}_{startts:%Y%m%d%H%M}_{endts:%Y%m%d%H%M}.csv"
    output_directory = create_folder(country)
    output_filename = f"{country}_{station}_{startts:%Y%m%d}_{endts:%Y%m%d}.csv"
    output_filepath = os.path.join(output_directory, output_filename)
    # Split the data into lines
    lines = data.split("\n")

    # Extract column names from the first line
    column_names = lines[0].split(",")

    # Initialize lists to store data
    data_rows = []

    # Iterate through the remaining lines (data rows)
    for line in lines[1:]:
        if line:  # Skip empty lines
            data_row = line.split(",")
            data_rows.append(data_row)

    # Create a DataFrame from the data rows using the extracted column names
    df = pd.DataFrame(data_rows, columns=column_names)

    # Save the DataFrame to a CSV file
    df.to_csv(output_filepath, index=False, encoding="utf-8")
    print(f'{output_filename} done!')

In [12]:
def download_and_save_data(url_site, country, station, startts, endts):
    start_time = time.time()  # Record start time
    data = download_data(url_site)
    end_time = time.time()  # Record end time
    download_time = end_time - start_time

    if data:
        save_data(url_site, country, station, startts, endts)
        print(f"{station} - Download time: {download_time:.3f} s")
    else:
        print(f"{station} - Download failed")

In [13]:
def download_and_save_data_thread(args):
    url_site, country, station_id, startts, endts = args
    download_and_save_data(url_site, country, station_id, startts, endts)

In [14]:
# UK example
country = [
    "GB",
]

data_folder = "data"
data_category = "raw_data"
output_folder = "ASOS_DATA"

In [ ]:
for i in range(1976,2023):
    start_date = datetime.datetime(i, 1, 1)
    end_date = datetime.datetime(i+1, 1, 1)

In [15]:
start_date = datetime.datetime(1976, 1, 1)
end_date = datetime.datetime(2023, 1, 1)

In [16]:
gb_df = get_all_station_by_network(country)

Folder '/Users/ww721/JupyterNotebookPath/IRP_20220602/irp_ww721_bakcup/src/notebooks/GB_ASOS_DATA' already exists.


In [17]:
gb_df

,Name,ID,Latitude,Logitude,Elevation,Country,Network,Archive_Begin,Archive_End,Time_Domain
0,Aberdeen,EGPD,57.2049,-2.2053,65.000000,GB,GB__ASOS,1928-01-01,None,(1928-Now)
1,Alderney,EGJA,49.7061,-2.2147,71.000000,GB,GB__ASOS,1987-06-29,None,(1987-Now)
2,Angelsey,EGOV,53.2527,-4.5365,11.000000,GB,GB__ASOS,1928-01-01,None,(1928-Now)
3,Barkston Heath,EGYE,52.9622,-0.5616,112.000000,GB,GB__ASOS,1995-11-29,None,(1995-Now)
4,Belfast,EGAC,54.6135,-5.8735,5.000000,GB,GB__ASOS,1973-01-01,None,(1973-Now)
...,...,...,...,...,...,...,...,...,...,...
107,Wittering,EGXT,52.6114,-0.4612,84.000000,GB,GB__ASOS,1973-02-28,None,(1973-Now)
108,Woodvale,EGOW,53.5816,-3.0555,12.000000,GB,GB__ASOS,2005-06-20,None,(2005-Now)
109,Wyton,EGUY,52.3572,-0.1078,41.000000,GB,GB__ASOS,1950-10-02,2019-11-18,(1950-2019)
110,Yeovilton,EGDY,51.0063,-2.6428,23.000000,GB,GB__ASOS,1972-12-31,None,(1972-Now)


In [18]:
url_site_list, id_list = get_data_url(gb_df, start_date, end_date)

In [19]:
args_list = [
    (url_site, "GB", station_id, start_date, end_date)
    for url_site, station_id in zip(url_site_list, id_list)
]

with concurrent.futures.ThreadPoolExecutor(max_workers=2) as executor:
    executor.map(download_and_save_data_thread, args_list)  # fast
    time.sleep(10)  # 

Folder '/Users/ww721/JupyterNotebookPath/IRP_20220602/irp_ww721_bakcup/src/notebooks/GB_ASOS_DATA' already exists.
GB_EGJA_19760101_20230101.csv done!
EGJA - Download time: 10.705 s
Folder '/Users/ww721/JupyterNotebookPath/IRP_20220602/irp_ww721_bakcup/src/notebooks/GB_ASOS_DATA' already exists.
GB_EGOV_19760101_20230101.csv done!
EGOV - Download time: 9.162 s
Folder '/Users/ww721/JupyterNotebookPath/IRP_20220602/irp_ww721_bakcup/src/notebooks/GB_ASOS_DATA' already exists.
GB_EGYE_19760101_20230101.csv done!
EGYE - Download time: 2.118 s
Folder '/Users/ww721/JupyterNotebookPath/IRP_20220602/irp_ww721_bakcup/src/notebooks/GB_ASOS_DATA' already exists.
GB_EGAC_19760101_20230101.csv done!
EGAC - Download time: 6.284 s
Folder '/Users/ww721/JupyterNotebookPath/IRP_20220602/irp_ww721_bakcup/src/notebooks/GB_ASOS_DATA' already exists.
GB_EGPD_19760101_20230101.csv done!
EGPD - Download time: 61.817 s
Folder '/Users/ww721/JupyterNotebookPath/IRP_20220602/irp_ww721_bakcup/src/notebooks/GB_ASOS_

In [ ]:
# args_list = []

# for i in range(1976, 2023):
#     start_date = datetime.datetime(i, 1, 1)
#     end_date = datetime.datetime(i+1, 1, 1)
#     for url_site, station_id in zip(url_site_list, id_list):
#         args_list.append((url_site, "GB", station_id, start_date, end_date))

# with concurrent.futures.ThreadPoolExecutor() as executor:
#     executor.map(download_and_save_data_thread, args_list)  # fast


In [98]:
country = ["GB",]
url_site_list=[]
id_list=[] 
start_date =  datetime.datetime(2022,1,1)
end_date =  datetime.datetime(2023,8, 2)

gb_df = get_all_station_by_network(country)
url_site_list,id_list = get_data_url(gb_df,start_date,end_date)
for url_site, station_id in tqdm(zip(url_site_list, id_list)):
    start_time = time.time()  # Record start time
    save_data(url_site, "GB",station_id, start_date, end_date)
    end_time = time.time()  # Record end time
    download_time = end_time - start_time
    print(f'Download time: {download_time:.3f} s')

1it [00:19, 19.91s/it]

GB_EGPD_20220101_20230802.csv done!
Download time: 19.910 s


2it [00:30, 14.59s/it]

GB_EGJA_20220101_20230802.csv done!
Download time: 10.869 s


3it [00:43, 13.60s/it]

...

GB_EGUY_20220101_20230802.csv done!
Download time: 0.454 s


111it [24:33, 11.47s/it]

GB_EGDY_20220101_20230802.csv done!
Download time: 21.984 s


112it [24:52, 13.33s/it]

GB_EGSY_20220101_20230802.csv done!
Download time: 19.512 s


In [11]:
def download_and_save_data(url_site, country, station, startts, endts):
    start_time = time.time()  # Record start time
    data = download_data(url_site)
    end_time = time.time()  # Record end time
    download_time = end_time - start_time

    if data:
        save_data(url_site, country, station, startts, endts)
        print(f'{station} - Download time: {download_time:.3f} s')
    else:
        print(f'{station} - Download failed')

def download_and_save_data_thread(args):
    url_site, country, station_id, startts, endts = args
    download_and_save_data(url_site, country, station_id, startts, endts)

In [ ]:
# UK example
country = [
    "GB",
]

start_date = datetime.datetime(2022, 1, 1)
end_date = datetime.datetime(2023, 8, 2)

gb_df = get_all_station_by_network(country)
url_site_list, id_list = get_data_url(gb_df, start_date, end_date)
args_list = [
    (url_site, "GB", station_id, start_date, end_date)
    for url_site, station_id in zip(url_site_list, id_list)
]

with concurrent.futures.ThreadPoolExecutor() as executor:
    executor.map(download_and_save_data_thread, args_list)

In [22]:
# UK example
country = [
    "GB",
]

start_date = datetime.datetime(2022, 1, 1)
end_date = datetime.datetime(2023, 8, 2)

gb_df = get_all_station_by_network(country)

Folder '/Users/ww721/JupyterNotebookPath/IRP_20220602/irp_ww721_bakcup/data/GB_ASOS_DATA' already exists.
